In [1]:
import requests
from bs4 import BeautifulSoup
import zipfile
import os
import time

In [3]:


# Fetch the webpage
url = "https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/"
response = requests.get(url)

# Parse the HTML
soup = BeautifulSoup(response.text, 'html.parser')

# Find all links to ZIP files
zip_links = [a['href'] for a in soup.find_all('a') if a['href'].endswith('.zip')]

for link in zip_links:
    # Correct the URL construction
    full_url = "https://nemweb.com.au" + link
    file_name = link.split('/')[-1]
    
    base = "./Actual_Roof"
    file_path = os.path.join(base, file_name)
    print(f"Attempting to download: {full_url}")
    
    r = requests.get(full_url, stream=True)

    with open(file_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    
    
        
        
    

Attempting to download: https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230720.zip
Attempting to download: https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230727.zip
Attempting to download: https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230803.zip
Attempting to download: https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230810.zip
Attempting to download: https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230817.zip
Attempting to download: https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230824.zip
Attempting to download: https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230831.zip
Attempting to download: https://nemweb.com.au/Reports/Archive/ROOFTOP_PV/ACTUAL/PUBLIC_ROO

In [4]:
from zipfile import ZipFile
files = os.listdir("./Actual_Roof")
zip_files = [f for f in files if f.endswith('.zip')]
zip_files.sort()
print(zip_files)


for file in zip_files:
    file = os.path.join("./Actual_Roof" , file)
    date_str = file.split('_')[-1].replace('.zip', '')

    new_dir = f"./Actual_Roof_sub/{date_str}"
    os.makedirs(new_dir,exist_ok=True)
    with zipfile.ZipFile(file, 'r') as zip_ref:
        zip_ref.extractall(new_dir)
    

['PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230720.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230727.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230803.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230810.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230817.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230824.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230831.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230907.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230914.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230921.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20230928.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20231005.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20231012.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20231019.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20231026.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20231102.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20231109.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_20231116.zip', 'PUBLIC_ROOFTOP_PV_ACTUAL_MEASUREMENT_2023112

In [ ]:
subFolders = os.listdir("./Actual_Roof_sub")
subFolders.sort()

print(subFolders)
for folder in subFolders:
    if "DS_Store" in folder:
        continue
    
    base = os.path.join("./Actual_Roof_sub",folder)
    files = os.listdir(base)
    
    files.sort()
    new_dir = f"./Actual_Roof_sub/{folder}_csv"
    
    os.makedirs(new_dir,exist_ok=True)
    for file in files:
        
        file = os.path.join(base,file)
        print(file)
        with zipfile.ZipFile(file, 'r') as zip_ref:
            zip_ref.extractall(new_dir)

In [ ]:
import pandas as pd
import numpy as np
import os
base = "./Actual_Roof_sub"
csvfolders = os.listdir(base)
csvfolders = [f for f in csvfolders if "csv" in f]
csvfolders.sort()
target_regions = ["QLD1","NSW1","SA1","VIC1","TAS1"]
for csvfolder in csvfolders:
    folderpath = os.path.join(base,csvfolder)
    csvfiles = os.listdir(folderpath)
    csvfiles.sort()
    for file in csvfiles:
        folder_path = os.path.join(base,csvfolder)
        file_path = os.path.join(folder_path,file)
        
        

        df = pd.read_csv(file_path,skiprows=1, header=0,index_col=False)
        
        new_df = df[['INTERVAL_DATETIME', 'REGIONID', 'POWER', 'TYPE', 'LASTCHANGED']].copy()
        

        new_df.columns = ['Date', 'Region', 'OPOWER',"TYPE","LASTCHANGED"]
        new_df = new_df.dropna()
        
        new_df = new_df[new_df['Region'].isin(target_regions)]

        new_df = new_df[new_df["TYPE"]=="MEASUREMENT"]

        if new_df is None:
            continue
        else:
            new_df.drop("TYPE",axis=1,inplace=True)
        if os.path.isfile("Actual_Roof.csv"):
            

            new_df.to_csv("Actual_Roof.csv", mode='a', header=False, index=False)
        else:
            new_df.to_csv("Actual_Roof.csv", mode='w', header=True, index=False)
        
        
        
    
    
